## Анализ потребительской лояльности

__*Цель исследования*__ - определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России.

Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ.

Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».
Компания провела опрос и попросила вас подготовить дашборд с его итогами. Большую базу данных для такой задачи разворачивать не стали и выгрузили данные в SQLite.

Чтобы оценить результаты опроса, оценки обычно делят на три группы:
* 9-10 баллов — «cторонники» (англ. promoters);
* 7-8 баллов — «нейтралы» (англ. passives);
* 0-6 баллов — «критики» (англ. detractors).

__*Итоговое значение NPS рассчитывается по формуле: % «сторонников» - % «критиков».*__

Таким образом, значение этого показателя варьируется от -100% (когда все клиенты «критики») до 100% (когда все клиенты лояльны к сервису). Но это крайние случаи, которые редко встретишь на практике.
Интерпретируя результаты NPS-опросов, следует также помнить, что само значение мало о чём говорит. Однако исследования показывают, что клиенты-сторонники полезны любому бизнесу. Они чаще других повторно совершают покупки, активнее тестируют обновления и приводят в сервис своих друзей и знакомых. Поэтому NPS остаётся одной из важнейших метрик бизнеса.


In [1]:
import pandas as pd

In [2]:
from sqlalchemy import create_engine

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
try:
    path_to_db = './datasets/telecomm_csi.db' # путь к файлу
except FileNotFoundError:
    print('Файл не найден')

In [ ]:
query = """
SELECT u.user_id,
       u.lt_day,
       CASE
           WHEN u.lt_day <= 365 THEN 'True'
           ELSE 'False'
       END AS is_new,
       u.age,
       u.gender_segment,
       u.os_name,
       u.cpe_type_name,
       l.country,
       l.city,
       ags.bucket_min,
       ags.bucket_max,
       substr(ags.title,3) AS name_age_segment,
       ts.bucket_min,
       ts.bucket_max,
       substr(ts.title,3) AS name_traffic_segment,
       ls.bucket_min,
       ls.bucket_max,
       substr(ls.title,3) AS name_lifetime_segment,
       u.nps_score,
       CASE
           WHEN u.nps_score >= 9 THEN 'promoters'
           WHEN u.nps_score >= 7 THEN 'passives'
           ELSE 'detractors'
       END AS nps_group    
FROM user AS u
LEFT JOIN location AS l ON u.location_id=l.location_id
LEFT JOIN age_segment AS ags ON u.age_gr_id=ags.age_gr_id
LEFT JOIN traffic_segment AS ts ON u.tr_gr_id=ts.tr_gr_id
LEFT JOIN lifetime_segment AS ls ON u.lt_gr_id=ls.lt_gr_id

"""


In [8]:
df = pd.read_sql(query, engine)

In [9]:
df

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,bucket_min,bucket_max,name_age_segment,bucket_min,bucket_max,name_traffic_segment,bucket_min,bucket_max,name_lifetime_segment,nps_score,nps_group
0,A001A2,2320,False,45.0,1.0,ANDROID,SMARTPHONE,Россия,Уфа,45.0,54.0,45-54,1.0,5.0,1-5,36.0,NaN,36+,10,promoters
1,A001WF,2344,False,53.0,0.0,ANDROID,SMARTPHONE,Россия,Киров,45.0,54.0,45-54,1.0,5.0,1-5,36.0,NaN,36+,10,promoters
2,A003Q7,467,False,57.0,0.0,ANDROID,SMARTPHONE,Россия,Москва,55.0,64.0,55-64,20.0,25.0,20-25,13.0,24.0,13-24,10,promoters
3,A004TB,4190,False,44.0,1.0,IOS,SMARTPHONE,Россия,РостовнаДону,35.0,44.0,35-44,0.1,1.0,0.1-1,36.0,NaN,36+,10,promoters
4,A004XT,1163,False,24.0,0.0,ANDROID,SMARTPHONE,Россия,Рязань,16.0,24.0,16-24,5.0,10.0,5-10,36.0,NaN,36+,10,promoters
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502488,ZZZKLD,1249,False,54.0,1.0,ANDROID,SMARTPHONE,Россия,Москва,45.0,54.0,45-54,1.0,5.0,1-5,36.0,NaN,36+,5,detractors
502489,ZZZLWY,129,True,31.0,0.0,ANDROID,SMARTPHONE,Россия,Москва,25.0,34.0,25-34,1.0,5.0,1-5,4.0,6.0,4-6,8,passives
502490,ZZZQ5F,522,False,36.0,0.0,ANDROID,SMARTPHONE,Россия,Сургут,35.0,44.0,35-44,25.0,30.0,25-30,13.0,24.0,13-24,10,promoters
502491,ZZZQ8E,2936,False,37.0,1.0,ANDROID,SMARTPHONE,Россия,УланУдэ,35.0,44.0,35-44,65.0,70.0,65-70,36.0,NaN,36+,9,promoters


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 20 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   user_id                502493 non-null  object 
 1   lt_day                 502493 non-null  int64  
 2   is_new                 502493 non-null  object 
 3   age                    501939 non-null  float64
 4   gender_segment         501192 non-null  float64
 5   os_name                502493 non-null  object 
 6   cpe_type_name          502493 non-null  object 
 7   country                502493 non-null  object 
 8   city                   502493 non-null  object 
 9   bucket_min             501254 non-null  float64
 10  bucket_max             488703 non-null  float64
 11  name_age_segment       502493 non-null  object 
 12  bucket_min             502493 non-null  float64
 13  bucket_max             491388 non-null  float64
 14  name_traffic_segment   502493 non-nu

In [11]:
df.isnull().sum()

user_id                       0
lt_day                        0
is_new                        0
age                         554
gender_segment             1301
os_name                       0
cpe_type_name                 0
country                       0
city                          0
bucket_min                 1239
bucket_max                13790
name_age_segment              0
bucket_min                    0
bucket_max                11105
name_traffic_segment          0
bucket_min                    0
bucket_max               274299
name_lifetime_segment         0
nps_score                     0
nps_group                     0
dtype: int64

Мы видем большое количество пропусков в столбцах со значениями границ сегментов, изучив содержание этих столбцов и столбцов с наименованием сегментов, можно сделать вывод, что все необходимые значения можно увидеть в столбце с наименованием, поэтому мы избавимся от ненужных столбцов.

In [12]:
df = df.drop(columns=['bucket_min','bucket_max','bucket_min','bucket_max','bucket_min','bucket_max'])

In [13]:
df

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,name_age_segment,name_traffic_segment,name_lifetime_segment,nps_score,nps_group
0,A001A2,2320,False,45.0,1.0,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,promoters
1,A001WF,2344,False,53.0,0.0,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,promoters
2,A003Q7,467,False,57.0,0.0,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,promoters
3,A004TB,4190,False,44.0,1.0,IOS,SMARTPHONE,Россия,РостовнаДону,35-44,0.1-1,36+,10,promoters
4,A004XT,1163,False,24.0,0.0,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,promoters
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502488,ZZZKLD,1249,False,54.0,1.0,ANDROID,SMARTPHONE,Россия,Москва,45-54,1-5,36+,5,detractors
502489,ZZZLWY,129,True,31.0,0.0,ANDROID,SMARTPHONE,Россия,Москва,25-34,1-5,4-6,8,passives
502490,ZZZQ5F,522,False,36.0,0.0,ANDROID,SMARTPHONE,Россия,Сургут,35-44,25-30,13-24,10,promoters
502491,ZZZQ8E,2936,False,37.0,1.0,ANDROID,SMARTPHONE,Россия,УланУдэ,35-44,65-70,36+,9,promoters


Удалим так же строки с пропущенными значениями столбцов `age` и `gender_segment` так как ввиду небольшого количества пропусков, на результаты исследования это не скажется.

In [14]:
df = df.dropna(subset=['age','gender_segment']) # удалим строки с пропущенными значениями в столбцах

In [15]:
df.isnull().sum()

user_id                  0
lt_day                   0
is_new                   0
age                      0
gender_segment           0
os_name                  0
cpe_type_name            0
country                  0
city                     0
name_age_segment         0
name_traffic_segment     0
name_lifetime_segment    0
nps_score                0
nps_group                0
dtype: int64

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 501152 entries, 0 to 502492
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   user_id                501152 non-null  object 
 1   lt_day                 501152 non-null  int64  
 2   is_new                 501152 non-null  object 
 3   age                    501152 non-null  float64
 4   gender_segment         501152 non-null  float64
 5   os_name                501152 non-null  object 
 6   cpe_type_name          501152 non-null  object 
 7   country                501152 non-null  object 
 8   city                   501152 non-null  object 
 9   name_age_segment       501152 non-null  object 
 10  name_traffic_segment   501152 non-null  object 
 11  name_lifetime_segment  501152 non-null  object 
 12  nps_score              501152 non-null  int64  
 13  nps_group              501152 non-null  object 
dtypes: float64(2), int64(2), object(10)


In [17]:
df['age'] = pd.to_numeric(df['age'], downcast='integer')

In [18]:
df['gender_segment'] = pd.to_numeric(df['gender_segment'], downcast='integer')

In [19]:
df

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,name_age_segment,name_traffic_segment,name_lifetime_segment,nps_score,nps_group
0,A001A2,2320,False,45,1,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,promoters
1,A001WF,2344,False,53,0,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,promoters
2,A003Q7,467,False,57,0,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,promoters
3,A004TB,4190,False,44,1,IOS,SMARTPHONE,Россия,РостовнаДону,35-44,0.1-1,36+,10,promoters
4,A004XT,1163,False,24,0,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,promoters
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502488,ZZZKLD,1249,False,54,1,ANDROID,SMARTPHONE,Россия,Москва,45-54,1-5,36+,5,detractors
502489,ZZZLWY,129,True,31,0,ANDROID,SMARTPHONE,Россия,Москва,25-34,1-5,4-6,8,passives
502490,ZZZQ5F,522,False,36,0,ANDROID,SMARTPHONE,Россия,Сургут,35-44,25-30,13-24,10,promoters
502491,ZZZQ8E,2936,False,37,1,ANDROID,SMARTPHONE,Россия,УланУдэ,35-44,65-70,36+,9,promoters


Просмотрим уникальные значения столбцов.

In [20]:
columns = ['age','gender_segment','os_name','cpe_type_name','country','city','name_age_segment', 'name_traffic_segment','name_lifetime_segment']

In [21]:
for column in columns:
    print(df[column].unique())

[45 53 57 44 24 42 35 36 54 39 21 27 60 34 47 37 43 33 31 25 51 28 41 40
 46 48 32 30 52 59 26 50 62 29 55 22 38 56 23 49 66 74 75 17 65 64 69 58
 20 19 80 70 81 63 67 68 72 15 79 18 73 14 71 61 16 77 13 76 10 78 12 82
 11 83 89 84 85 87 86]
[1 0]
['ANDROID' 'IOS' 'OTHER' 'unknown' 'PROPRIETARY' 'WINDOWS PHONE'
 'SYMBIAN OS' 'BADA OS' 'WINDOWS MOBILE']
['SMARTPHONE' 'TABLET' 'MOBILE PHONE/FEATURE PHONE' 'PHONE' 'ROUTER'
 'MODEM' 'WLAN ROUTER' 'USB MODEM' 'unknown' 'PORTABLE(INCLUDE PDA)'
 'NETWORK DEVICE' 'MIFI ROUTER' 'MOBILE TEST PLATFORM' 'HANDHELD']
['Россия']
['Уфа' 'Киров' 'Москва' 'РостовнаДону' 'Рязань' 'Омск' 'СанктПетербург'
 'Волгоград' 'Тольятти' 'Казань' 'Самара' 'Красноярск' 'Екатеринбург'
 'Калуга' 'Краснодар' 'Иркутск' 'Пермь' 'Владимир' 'Ижевск' 'Тюмень'
 'Оренбург' 'НижнийНовгород' 'Брянск' 'Челябинск' 'Астрахань' 'Сургут'
 'Тверь' 'Новосибирск' 'НабережныеЧелны' 'Махачкала' 'Воронеж' 'Курск'
 'Владивосток' 'Балашиха' 'Пенза' 'Калининград' 'Тула' 'Саратов'
 'Кемерово'

In [22]:
df.duplicated().sum()

0

In [23]:
df['gender_segment'] = df['gender_segment'].astype('object')

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 501152 entries, 0 to 502492
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   user_id                501152 non-null  object
 1   lt_day                 501152 non-null  int64 
 2   is_new                 501152 non-null  object
 3   age                    501152 non-null  int8  
 4   gender_segment         501152 non-null  object
 5   os_name                501152 non-null  object
 6   cpe_type_name          501152 non-null  object
 7   country                501152 non-null  object
 8   city                   501152 non-null  object
 9   name_age_segment       501152 non-null  object
 10  name_traffic_segment   501152 non-null  object
 11  name_lifetime_segment  501152 non-null  object
 12  nps_score              501152 non-null  int64 
 13  nps_group              501152 non-null  object
dtypes: int64(2), int8(1), object(11)
memory usage: 54.0+

In [25]:
dash_nps = df

In [26]:
dash_nps.to_csv('dash_nps2.csv')

* Ссылка на презентацию: https://drive.google.com/file/d/1ys5YGtJowOi9-0yEjxA4m2Wx7N_9BMc-/view?usp=sharing
* Сылка на дашборды: https://public.tableau.com/views/dash_nps/Dashboard1?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link

In [27]:
#CODE REVIEW
# Формируем новый столбец по такому алгоритму:
# if promoters then 1, if detractors then -1, else 0
df["nps_group"].unique()

array(['promoters', 'detractors', 'passives'], dtype=object)

In [28]:
#CODE REVIEW
# Алгоритм реализую словарем, но можно и функцией
df["total_nps"] = df["nps_group"].map({
    "promoters": 1,
    "detractors": -1,
    "passives": 0
})

Такой подход дает возможность для любых срезов, фильтраций, агрегаций считать среднее по столбцу <code>total_nps</code> и получать готовый уровень NPS, попробуем парочку примеров👇

In [29]:
#CODE REVIEW
# Проверим общий NPS
total_nps = df["total_nps"].mean()
print(f"Total NPS: {total_nps:.1%}")

Total NPS: 22.0%


In [30]:
#CODE REVIEW
# Например, сводная таблица по полу и возрасту
df.pivot_table(index="gender_segment", columns="name_age_segment", values="total_nps", aggfunc="mean")\
    .style.format('{:.1%}')

name_age_segment,16-24,25-34,35-44,45-54,55-64,66 +,до 16
gender_segment,,,,,,,
0,3.6%,7.9%,19.4%,28.7%,37.3%,45.1%,-10.0%
1,6.2%,13.2%,24.7%,34.9%,45.3%,54.8%,1.9%


In [31]:
#CODE REVIEW
# Например, сводная таблица по полу
df.pivot_table(index="gender_segment", values="total_nps", aggfunc="mean")\
    .style.format('{:.1%}')

,total_nps
gender_segment,
0,17.7%
1,25.6%


In [32]:
#CODE REVIEW
# И что-нибудь большое, сводная таблица по возрасту и lifetime
df.pivot_table(index="name_age_segment", columns="name_lifetime_segment", values="total_nps", aggfunc="mean")\
    .style.format('{:.1%}', na_rep="")

name_lifetime_segment,1,13-24,2,25-36,3,36+,4-6,7-12
name_age_segment,,,,,,,,
16-24,,7.2%,21.2%,-0.6%,22.4%,-8.4%,23.9%,17.7%
25-34,50.0%,18.2%,30.0%,11.6%,32.6%,-0.5%,29.7%,26.4%
35-44,33.3%,32.1%,41.2%,24.8%,41.7%,14.2%,41.6%,38.0%
45-54,100.0%,41.0%,52.5%,33.8%,54.7%,27.0%,49.3%,46.9%
55-64,100.0%,48.4%,55.7%,44.4%,54.9%,38.9%,58.0%,53.8%
66 +,,53.3%,60.4%,49.9%,60.0%,50.6%,55.2%,60.2%
до 16,,3.1%,16.7%,-0.8%,26.1%,-19.5%,14.3%,8.5%


<hr style="border: 2px solid orange;" />

In [33]:
df.pivot_table(index="name_lifetime_segment", values="total_nps", aggfunc="mean")\
    .style.format('{:.1%}')

,total_nps
name_lifetime_segment,
1,61.5%
13-24,28.1%
2,36.6%
25-36,22.2%
3,38.1%
36+,15.8%
4-6,37.1%
7-12,34.2%


In [34]:
df.pivot_table(index="name_age_segment", values="total_nps", aggfunc="mean")\
    .style.format('{:.1%}')

,total_nps
name_age_segment,
16-24,4.8%
25-34,10.6%
35-44,22.2%
45-54,32.3%
55-64,42.5%
66 +,51.6%
до 16,-4.4%
